In [1]:
import torch 
from torch import nn
from torch.nn import functional as F
from GetInit import GetInit
from GetLoader import GetLoader
from ModelEmbedding import ModelEmbedding
from MyDataset import MyDataset
from TrainFunc import TrainFunc
from TextRNN import TextRNN

In [2]:
data_root = {
    "train_path": '../../data/train_torch.csv',
    "test_path": "../../data/test_a.csv",
    "sub_path": "../../data/test_a_sample_submit.csv",
    "w2v_path": "../../data/word2vec.bin"
}
config = GetInit(data_root)
model_embedding = ModelEmbedding(data_root["w2v_path"])

train_dataset = MyDataset(model_embedding,
                          corpus=config.x_train,
                          corpus_label=config.y_train,
                          with_label=True,
                          max_length=600
                         )
test_dataset = MyDataset(model_embedding,
                         corpus=config.x_test,
                         with_label=False,
                         max_length=600
                        )
loader = GetLoader(train_dataset, test_dataset)
del train_dataset
del test_dataset

GetInit Start!
GetInit End!
ModelEmbedding End!
GetLoader End


In [3]:
# 建立model
model=TextRNN(model_embedding.dict_length, embedding_dim=300,hidden_size=50, output_size=14, dropout=0.5)
model.init_weights(model_embedding.embedding, is_static=False)
model = model.cuda()
criterion = nn.NLLLoss()
# opt = torch.optim.Adam(model.parameters(), lr=3e-4)
# opt = torch.optim.Adagrad(model.parameters(), lr=1e-2)
opt = torch.optim.RMSprop(model.parameters(), lr=1e-3)
# opt = torch.optim.SGD(model.parameters(), lr=1e-3)
# 开始训练
mytrain = TrainFunc(model, criterion, opt, loader.train_loader, loader.valid_loader, loader.test_loader)
best_model = mytrain.train(25)

Train Epoch: 1 [0/180000 (0%)]	Loss: 2.637796
Train Epoch: 1 [12800/180000 (7%)]	Loss: 1.495497
Train Epoch: 1 [25600/180000 (14%)]	Loss: 1.643173
Train Epoch: 1 [38400/180000 (21%)]	Loss: 1.731077
Train Epoch: 1 [51200/180000 (28%)]	Loss: 1.466582
Train Epoch: 1 [64000/180000 (36%)]	Loss: 1.016037
Train Epoch: 1 [76800/180000 (43%)]	Loss: 0.922734
Train Epoch: 1 [89600/180000 (50%)]	Loss: 1.079167
Train Epoch: 1 [102400/180000 (57%)]	Loss: 0.724420
Train Epoch: 1 [115200/180000 (64%)]	Loss: 0.713276
Train Epoch: 1 [128000/180000 (71%)]	Loss: 0.808043
Train Epoch: 1 [140800/180000 (78%)]	Loss: 0.768560
Train Epoch: 1 [153600/180000 (85%)]	Loss: 0.665367
Train Epoch: 1 [166400/180000 (92%)]	Loss: 0.647880
Train Epoch: 1 [179200/180000 (100%)]	Loss: 0.655765
	Loss: 0.9963(train)	|	Acc: 69.2%(train)
	Loss: 0.5992(valid)	|	Acc: 82.2%(valid)
	Micro: 0.8225(valid)	|	Macro: 0.6920(valid)
Now_best:0.6920
Train Epoch: 2 [0/180000 (0%)]	Loss: 0.672198
Train Epoch: 2 [12800/180000 (7%)]	Loss: 0.6

In [4]:
import pandas as pd
import numpy as np
df_sub=pd.read_csv(data_root["sub_path"])
df_sub.label=np.array(ans)
df_sub.label=df_sub.label.astype("int64")
save_name="./textrnn2_{:.4f}.csv".format(mytrain.best_score)
df_sub.to_csv(save_name, index=False)

NameError: name 'ans' is not defined

In [ ]:
exit()